In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
EPISODES = 1000

In [3]:
class DQNAgent:
    def __init__(self,state_size,action_size):
        self.state_size=state_size
        self.action_size=action_size
        self.memory = deque(maxlen=2000)
        self.gamma=0.95 #discount rate
        self.epsilon = 1.0 #exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay=0.995
        self.learning_rate= 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        model=Sequential()
        model.add(Dense(24,input_dim = self.state_size, activation='relu'))
        model.add(Dense(24,activation='relu'))
        model.add(Dense(self.action_size,activation='linear')) #output layer of number of actions that can be taken i.e left or right
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
            self.memory.append((state, action, reward, next_state, done))

    def act(self,state,explore=True):
        if explore and np.random.rand()<= self.epsilon:
                    return random.randrange(self.action_size) #randomly decide an action
        act_values = self.model.predict(state) #predict using the network which will give back the expected reward of all actions
        return np.argmax(act_values[0]) #returns action

    def replay(self,batch_size):
        #take a sample of a random subset of all states remembered
        minibatch = random.sample(self.memory,batch_size)
        for state,action, reward, next_state,done in minibatch:
            target=reward
            #compute the reward and future expected reward as the target value
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            #force training of only the single action
            #by setting the target for all other actions
            #as the same value the network produced
            target_f = self.model.predict(state)
            #and set the action we want to train to the real value
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0)
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

In [4]:
def load(self,name):
        delf.model.load_weights(name)

In [ ]:
if __name__=="__main__":
    env=gym.make('CartPole-v1')
    #get state size
    state_size=env.observation_space.shape[0]
    #get action size
    action_size=env.action_space.n
    #print(state_size),print(action_size)
    agent=DQNAgent(state_size,action_size)
    #agent.load("cartpole-dqn.h5")
    done=False
    batch_size=32
    
    for e in range(EPISODES):
        #reset the environment to get the initial state
        state=env.reset()
        state=np.reshape(state,[1,state_size])
        for time in range(500):
            #render visual
            env.render()
            #user the action value that either explores or chooses the best action
            action = agent.act(state,True)
            #take the action and get the values of the environment
            next_state,reward,done,_ = env.step(action)
            #get the state in the format that the network prefers
            next_state=np.reshape(next_state,[1,state_size])
            #remember what happened to retrain later
            agent.remember(state,action,reward,next_state,done)
            state=next_state
            #if done print information about the ultimate score
            if done:
                print("episode: {}/{}, score: {}, e: {:2}".format(e,EPISODES,time,agent.epsilon))
                break
            #if there are 32 or more examples in the memory then replace
            if len(agent.memory)>batch_size:
                agent.replay(batch_size)
                

In [9]:
state=env.reset()
print(state)

[ 0.00498662 -0.00867378  0.01053037  0.02939508]


In [13]:
act_values = DQNAgent.model.predict(state)

AttributeError: type object 'DQNAgent' has no attribute 'model'

In [ ]:
np.reshape(state,[1,state_size])